# **FolT Homework 6**


## Alexander Praus, Maike Arnold

In [ ]:
# Execute before and only in colab, if you use jupyter ignore this cell
!pip install --target=$nb_path nltk==3.5
!python3 -m nltk.downloader popular
!python3 -m nltk.downloader book

### Homework 6.1 (10 points)
Implement an SMS decoder.  Similar to the T9 system on mobile phones, your decoder should translate from digit sequences to words:


a) Choose at least one appropriate corpus and discuss,  why you chose this corpus. You will use the corpus to estimate which word is more frequent and should be a preferred output.

We will be using the nps_chat corpus. This corpus contains a collection of instant messaging chat sessions[^1]. Therefore, this corpus contains much more relevant texts than, for example, the Gutenberg corpus when working with informal text sources such as SMS. A chat corpus is also more likely to take into consideration abbreviations often used in SMS.

[^1]: https://www.nltk.org/book/ch02.html


b) Implement a function `get_T9_word(digits)` which for a given sequence of digits, e.g. "252473", returns themost likely word, e.g. "Claire"

In [ ]:
from nltk.corpus import nps_chat
from nltk.corpus import brown

In [ ]:
T9_dict = {0: ' ', 1:'.?!', 2: ['a','b','c'],3: ['d','e','f'],4: ['g','h','i'],5: ['j','k','l'],6: ['m','n','o'],7: ['p','q','r','s'],8: ['t','u','v'],9: ['w','x','y','z']}
# T9_dict = {2: ['a','b','c'],3: ['d','e','f'],4: ['g','h','i'],5: ['j','k','l'],6: ['m','n','o'],7: ['p','q','r','s'],8: ['t','u','v'],9: ['w','x','y','z']}

fq = nltk.FreqDist([word.lower() for word in nps_chat.words()])
# fq = nltk.FreqDist([word.lower() for word in brown.words()])

def get_T9_word(digits):
    words = []
    for digit in digits:
        if int(digit) not in T9_dict:
            return None
        if len(words) == 0:
            words.extend(T9_dict[int(digit)])
        else:
            words = list(set([a+b for a in words for b in T9_dict[int(digit)]]))
    prob = 0
    word = ''
    for pos in words:
        if fq[pos] > prob:
            prob = fq[pos]
            word = pos
    if prob != 0:
        return word

In [ ]:
# Test
print(get_T9_word('252473')) # claire
print(get_T9_word('4663'), get_T9_word('3836464')) # good evening
print(get_T9_word('9352663')) # welcome
print(get_T9_word('4373')) # here
print(get_T9_word('1111')) # no word

## Comment

It was not 100% clear to us what the encoding should look like. We assumed that all number 0-9 should be considered. This leads to the problem that `print(get_T9_word('1111')) # no word` is not true using out chosen corpus. However, when using for exampel the Brown corpus, we achieve a similiar result (to test this, simply toggle comments on the two lines with FreqDists). We chose to continue using the nps_chat corpus. This also affects the results in part (c). In this case the result is `hello my friend peter i an find` and the errors can be explained the same way the second error is explained in part (c).

c) Apply the decoder to each digit sequence in this “sentence”: 

`['43556','69','374363','73837','4','26','3463']`

The original sentence was: *“hello my friend peter i am fine”* 

Is the output readable?
What errors have been made?

In [ ]:
digits = ['43556','69','374363','73837','4','26','3463']
for digit in digits:
    print(get_T9_word(digit))

The output contains two mistakes.
1. Peter: there was no word found in the corpus that corresponds to the didgets making up the name Peter. The reason for this is most likely due to that fact that pronouns are likely relatively rare in the given corpus. Therefore, it is easy for a specific name to be missing.
2. fine: Insted of returning 'fine' the word 'find' was returned. This is simply due to the fact that the word 'find' is more common in the corpus and the fac that the method does not take context or grammatical correctness into consideration.

### Homework 6.2
This is a quite complicated optional task that you might explore if you are interested. It will not count as part of the official homework assignment.Improve the SMS decoder of homework 6.1 as follows:

a) Take the context into account, guess the word using the bigram probability of the previous entered word with the function `get_T9_word(prevW ord, number)`. Test the improvement with the `(context_word, digit)` tuples inthe following list:

In [ ]:
fq = nltk.FreqDist([word.lower() for word in brown.words()])
def get_all_T9_words(digits):
    words = []
    for digit in digits:
        if int(digit) not in T9_dict:
            return None
        if len(words) == 0:
            words.extend(T9_dict[int(digit)])
        else:
            words = list(set([a+b for a in words for b in T9_dict[int(digit)]]))
    possible = []
    for pos in words:
        if fq[pos]:
            possible.append(pos)
    return possible

In [ ]:
fq_bigrams = nltk.FreqDist([bigram for bigram in nltk.bigrams([word.lower() for word in brown.words()])])

def get_T9_word(prevWord, number):
    words = get_all_T9_words(number)
    prob = 0
    bigram = ''
    for word in words:
        if fq_bigrams[(prevWord, word)] > prob:
            prob = fq_bigrams[(prevWord, word)]
            bigram = word
    if prob != 0:
        return bigram
    else:
        for pos in words:
            if fq[pos] > prob:
                prob = fq[pos]
                bigram = pos
        return bigram


In [ ]:
# Test
print(get_T9_word('i',    '26'))
print(get_T9_word('its',  '26'))
print(get_T9_word('a',    '3463'))
print(get_T9_word('will', '3463'))
print(get_T9_word('the',  '1111'))

We decided to switch to the Brown corpus here because the chat corpus was not leading to very useful results.

(b)  Apply the decoder to each digit sequence in this “sentence”:

`['43556','69','374363','73837','4','26','3463']`

The original sentence was: *“hello my friend peter i am fine”* 

Is the output readable? What errors have been made?

In [ ]:
digits = ['43556','69','374363','73837','4','26','3463']

In [ ]:
sentence = ['']
for digit in digits:
    sentence.append(get_T9_word(sentence[len(sentence)-1], digit))
sentence = sentence [1:]
print(sentence)
    

This lead to a slight improvement. We now get 'am' instead of 'an' but the we still recieve 'find' instead of 'fine'